### Imports

In [1]:
import importlib
import os
from tqdm import tqdm

import findatree.io as io
import findatree.transformations as transformations
import findatree.interactive as interactive
import findatree.geo_to_image as geo_to_image
import findatree.segmentation as segmentation
import findatree.photometric as photometric

from bokeh.plotting import save
from bokeh.io import output_file

importlib.reload(io)
importlib.reload(transformations)
importlib.reload(interactive)
importlib.reload(geo_to_image)
importlib.reload(segmentation)
importlib.reload(photometric)

<module 'findatree.photometric' from 'C:\\Repos\\findatree\\findatree\\photometric.py'>

### Definitions

In [2]:
# Define full paths to directories containing dsm, dtm, ortho and shape-files 
dir_names=[]
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\DSMs'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\DTMs'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\Orthos'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\Kronen'])

# Define directories where return hdf5s and htmls are stored
save_dir_hdf5 = r"C:\Data\lwf\processed\2020\hdf5"
save_dir_html = r"C:\Data\lwf\processed\2020\html"

#### Processing

#### Prepare main loop

In [16]:
importlib.reload(io)

# Get all available tnr numbers in directory
tnrs = io.find_all_tnrs_in_dir(dir_names[0])

# Init list of exceptions
exception_list = []

#### Run main loop

In [23]:
for tnr in tqdm(tnrs):
    
    print()
    print(f"Processing: tnr_{tnr}")

    ################# [1] Load and save channels
    try:
        params_channels = {'tnr': tnr}
        channels, params_channels = geo_to_image.channels_load(dir_names, params_channels, verbose=False)
        io.channels_to_hdf5(channels, params_channels, dir_name = save_dir_hdf5)
    
    except:
        print(f"-> tnr_{tnr}: Exception during loading/saving of channels")
        exception_list.extend([tnr])

    ################# [2] Load and save human generated crowns with added photometric features
    try:
        crowns_human, params_crowns_human = io.load_shapefile(dir_names, params_channels, verbose=False)
        photometric.crowns_add_features(
            channels,
            params_channels,
            crowns_human,
            params_crowns_human,
        )
        io.crowns_to_hdf5(crowns_human, params_crowns_human, dir_name = save_dir_hdf5)
    
    except:
        print(f"-> tnr_{tnr}: Exception during loading/saving of crowns_human")
        exception_list.extend([tnr])

    ################# [3] Generate and save watershed crowns
    try:
        params_crowns_water = {}

        crowns_water, params_crowns_water = segmentation.watershed(
            channels,
            params_channels,
            params_crowns_water,
            verbose=False,
        )
        io.crowns_to_hdf5(crowns_water, params_crowns_water, dir_name = save_dir_hdf5)
    
    except:
        print(f"-> tnr_{tnr}: Exception during generation/saving of crowns_water")
        exception_list.extend([tnr])

    ################# [4] Create bokeh plot and save as html
    try:
        # Init Plotter object and adjust some attributes 
        plt = interactive.Plotter()
        plt.width = 500
        plt.channels_downscale = 1
        plt.show_features = ['id', 'ba', 'nbv', 'gilb', 'perc25_brightest_ndvi', 'median_brightest_ndvi', 'perc75_brightest_ndvi']

        # Add channels
        plt.add_channels(channels, params_channels)
        plt.figures_add_rgb()
        plt.figures_add_gray('chm')

        # Add crowns
        plt.togglers_add_crowns(crowns_water, params_crowns_water)
        plt.togglers_add_crowns(crowns_human, params_crowns_human)

        # Create and save layout
        layout = plt.create_layout()
        output_file(
            filename = os.path.join(save_dir_html, f"tnr{tnr}.html"),
            title = f"tnr{tnr}_{transformations.current_datetime()}",
            mode='inline',
            )
        save(layout)

    except:
        print(f"-> tnr_{tnr}: Exception during creation of html")
        exception_list.extend([tnr])


print()
print('Done!')
print(f"Exceptions occured @{exception_list}")

  0%|          | 0/261 [00:00<?, ?it/s]


Processing: tnr_11


  0%|          | 1/261 [00:31<2:18:08, 31.88s/it]


Processing: tnr_35
-> tnr_35: Exception during loading/saving of crowns_human


  1%|          | 2/261 [00:50<1:45:06, 24.35s/it]

...


Processing: tnr_17042
-> tnr_17042: Exception during loading/saving of crowns_human


100%|█████████▉| 260/261 [4:01:41<00:56, 56.49s/it]


Processing: tnr_17339
-> tnr_17339: Exception during loading/saving of crowns_human


100%|██████████| 261/261 [4:03:06<00:00, 55.89s/it]


Done!
Exceptions occured @[35, 73, 75, 79, 84, 86, 171, 176, 182, 184, 185, 188, 190, 192, 194, 460, 462, 477, 479, 485, 489, 879, 1356, 1381, 1898, 2386, 2399, 2411, 2413, 3398, 3418, 3930, 4396, 4412, 4436, 4444, 5441, 5930, 5946, 6430, 6466, 6470, 6474, 6478, 6969, 6977, 7475, 7511, 7521, 7525, 8492, 8494, 8498, 8532, 8973, 8979, 8981, 8989, 9025, 9464, 9468, 9472, 9477, 9489, 9493, 9509, 9515, 9517, 9948, 9958, 9966, 9973, 9976, 9980, 9984, 9986, 9988, 9996, 11062, 11068, 11576, 11581, 11597, 11601, 11612, 12153, 12161, 12169, 12179, 12545, 12694, 12711, 12715, 13212, 13229, 13259, 13825, 14272, 14293, 14309, 14327, 14723, 14727, 14877, 14879, 14887, 15024, 15027, 15046, 15303, 15311, 15313, 15319, 15327, 15531, 15537, 15746, 15974, 15998, 16181, 16195, 16197, 16201, 16412, 16636, 16645, 16649, 16657, 16829, 16833, 16838, 17019, 17032, 17034, 17042, 17339]
